### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2018 ###

# Hobby Farm Production Optimization #

#### Joel Atkins (jratkins@wisc.edu), Julin Peng (jpeng49@wisc.edu), and Junyi Wei (jwei53@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Our goal was to attempt to model the decisions facing a farmer when choosing what to grow on a hobby farm.  Given a plot of land, the farmer needs to choose which crops to grow, which livestock to raise and which equipment to purchase.  Unlike large-scale commercial farms, hobby farms tend to grow a diverse mixture of crops and livestock.  There is evidence that hobby farms fail more quickly than other types of agriculture[1], for many reasons but financial difficulties appear to be the primary driver.  Optimizing the profitability of such an endeavor, while also considering the constraint of diverisity in attempt to achieve a more "idyllic" setting, increases the likelihood of success.

The complexity of this optimization problem is higher than it may seem at first glance.  On a large scale monocultural farm the problem is relatively simple, which crop has the maximum profit per acre?  On a hobby farm with a goal of diversity this is not the case.  Often, a decision about one crop or livestock is affected by another.  For example, if a farmer decides to grow a crop which can be used as feed for their animals they decrease the possible profit from that crop while also decreasing costs associated with that livestock.

Finding data with which to model this problem is not difficult.  In fact, the challenge is actually in sifting through the abundance of government and industry reports in order to track down the various values.  As our goal was attempting to create an accurate model of this problem, continuing to add more detail to bring it closer and closer to reality, government sources were identified and values for costs and returns pulled from their tables [2][3][4][5][6][7][8][9][10].  If the focus of the work had been on searching for a true optimal set of choices and maximized return these data sources would have been further validated and the values normalized to ensure cross-compatibility (e.g dates, locations, etc).

The following section will describe the iterative process used to model this problem.  Starting with a simplied "toy" model and then adding complexity in order to bring the model closer to the real world problem.  The model description is followed by [Julia] code using [JuMP] (Julia for Mathematical Optimization) in order to encode the model and find optimal results.  These results are then thoroughly described, with the final model being compared to previous, simplied iterations.  Finally, conclusions about the problem, model and process are presented.

***Citations:***

[1] Moyer, J. (2015). What nobody told me about small farming: I can’t make a living | Salon https://www.salon.com/2015/02/10/what_nobody_told_me_about_small_farming_i_cant_make_a_living/

[2] U. S. Department of Agriculture. (2017). Crop Values 2016 Summary. Washington, DC: Author.

[3] U. S. Department of Agriculture. (2018). Crop Production 2017 Summary. Washington, DC: Author.

[4] Manitoba Agriculture Farm Management. (2017). Guidelines for Estimating Crop Production Costs. Winnipeg, Manitoba: Author.

[5] Iowa State University Extension and Outreach. (2018). Estimated Returns to Farrow to Finish. Ames, IA: Author.

[6] Iowa State University Extension and Outreach. (2018). Estimated Returns to Finishing Yearling Steers. Ames, IA: Author.

[7] Iowa State University Extension and Outreach. (2018). Estimated Returns to Finishing Steer Calves. Ames, IA: Author.

[8] Iowa State University Extension and Outreach. (2018). Agriculture Decision Maker. Retrieved from https://www.extension.iastate.edu/agdm/ldcostsreturns.html

[9] UW-Madison College of Agriculture and Life Sciences. (2003). Large-scale pastured poultry farming in the U.S. Madison, WI: Author.

[10] Hamra, C.F. (2010). An Assessment of the Potential Profitability of Poultry Farms: A Broiler Farm Feasibility Case Study. Master's Thesis. University of Tennessee at Martin. Martin, TN.

   [Julia]: <https://julialang.org/>
   [JuMP]: <https://github.com/JuliaOpt/JuMP.jl>


## 2. Mathematical model ##

<!---
As Joe see it, we should start with a description of the toy problem.  An LP which takes in the basic crop and livestock costs and returns and calculates the largest possible profit.

We then add a "diversity" constraint from MIP which ensures that the farm does not produce only one thing.  Maybe setting a minimum of 4 choices?  Or no more than 25% to a certain output?  These are just ideas.

Finally we try to connect the ouput of crops to the inputs required by livestock (i.e. if we grow grain we might save money by feeding that to our animals rather than selling it).

-->
### 2.1 v1: Toy Model ###

We used an iterative process to produce the model for our problem.  To begin we created a "toy" LP of the form:


$$\begin{aligned}
  \underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& cx \\
    \text{subject to:}\qquad& Ax \le b\\
    \end{aligned}$$


Such that $b$ is a vector of the parameters money, acres and housing available which define the specific instance of the problem.  $A$ is a matrix of monetary and space requirements for each crop/livestock choice.  And finally $c$ is a vector of profit values.

$$\begin{bmatrix} 
    a_{1,1} & \dots & a_{1,m+n} \\
    \vdots & \ddots & \vdots\\
    a_{3,1} & \dots  & a_{3,m+n} 
    \end{bmatrix}
    \begin{bmatrix} crop_1 \\ \vdots \\ crop_n \\ livestock_1 \\ \vdots \\ livestock_m \end{bmatrix} \leq
    \begin{bmatrix} acres\\ housing\\ money \end{bmatrix}$$
    $$Ax \geq b$$
    
Creating this toy model allowed us to code the problem without complex constraints in order to ensure that data import and indexing was working properly.  The issue with this model is that it is an oversimplification of the problem and will only choose the single highest return crop and livestock for available acreage and housing.

### 2.2 v2: Adding Diversity Constraint, now MIP ###

Adding detail, and complexity, to our model will bring it more in line with reality.  Adding a diversity constraint, i.e. forcing the optimum solution to include a minimum number of crop choices as well as a minimum number of livestock choices, changes our Linear Program into a mixed integer program.

In order to model this constraint we will indicator variables $z_i$ which indicate whether or not a certain crop or livestock is included in the optimum choice.

$$ crop_i > 0 \Rightarrow y_i = 1$$
$$ livestock_i > 0 \Rightarrow z_i = 1$$

As these are conditional statements they need to be converted to algebraic constraints.  This is done with the addition of a value $M$ which is the upper bound on an optimal solution.  

$$crop_i \leq My_i $$
$$livestock_i \leq Nz_i$$

In the case of crop decisions the upper bound was set to the input parameter for available fields, in the case of livestock decisions the upper bound was set to the input parameter for available money.  These new constraints ensure that the indicator variables are valid.  Additional constraints can then be added to enforce the diversity constraint using the indicator variables.  

$$ \sum_i y_i = \text{number of desired crops}$$
$$ \sum_i z_i = \text{number of desired livestock}$$

### 2.3 v3: Adding Balance ###



<!---
Sets:
1. I - a set of crops
2. J - a set of livestocks

Parameter:
1. tt_field - total acres of the field for crops 
2. tt_pasture - total acres of pasture for livestocks
3. tt_housing - total number of housing for poultry

Decision Variables:
1. Define variable crops[i] to indicate which and how much crops to be planted.
2. Define variable lstock[j] to indicate which and how many livestocks to raise.
3. Define binary variable x[i] to indicate whether crop i is selected, for i in I.
    $$xi =
    \begin{cases}
    1 & xi\ge 0\\
    0 & \text{ otherwise }
    \end{cases}$$
4. Define binary variable y[j] to indicate whether livestock j is selected, for j in J.
    $$yj =
    \begin{cases}
    1 & yj\ge 0\\
    0 & \text{ otherwise }
    \end{cases}$$

Constraints:
1. Availablity constraints:
    1. Total acres that planted crops must be no more than the available field
    2. Total acres that raise livestocks must be no more than the available pasture
    3. Total number of house that raise chickens must be no more than the available houses
2. Diversity constraints
    4. There must be (xi) crops.
    5. There must be (yj) livestocks.
3. 

Objective:
1. Maximize the total returns 
2. Minimize the total costs

This is a MILP model.
            
A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$\begin{bmatrix}
      1 & 2 \\
       3 & 4
    \end{bmatrix}
    \begin{bmatrix} x \\ y \end{bmatrix} =
    \begin{bmatrix} 5 \\ 6 \end{bmatrix}$$

And here is an example of an optimization problem in standard form:
$$\begin{aligned}
  \underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
    \text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
    & h_j(x) = 0 && j=1,\dots,r
    \end{aligned}$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).
-->

## 3. Solution ##

The following code represents our iterative process of creating a realistic model.  Each version implemented built upon the previous version.  The output of each model was analyzed to determine where it differed from an ideal model.  This was followed by adding complexity in order to better model the desired problem.

### 3.1 Preparation: Data Import ###

Data was collected from various sources (as outlined in section 1) and input into 2 CSV files, one for crops and one for livestock.  These CSV files contain info on various costs, yields and returns for the possible crop and livestock choices a farmer can make.  These various values are consolidated into columns which provide costs and gross returns for each of the possible choices.  This information is imported into well formatted arrays with the following code:

In [15]:
using DataFrames, CSV, NamedArrays
raw = CSV.read("crops1.csv");
# turn Crops into an array
crops_array = convert(Array,raw);
# the `names' of the DataFrame (header) are the information of each crop;
info_crops = names(raw[2:end]);
# create a list of crops from the crops array
crops = crops_array[1:end,1];
# create a NamedArray that specifies the information of each crop
crops = convert(Array{Symbol},crops )
using NamedArrays
crops_matrix = crops_array[1:end,2:end]; # rows are crops, columns are information of crops
crops_information_array = NamedArray(crops_matrix, (crops, info_crops), ("crops","info_crops"));


raw1 = CSV.read("LiveStock.csv");
# turn LiveStock into an array
lstocks_array = convert(Array,raw1);
# the `names' of the DataFrame (header) are the information of each livestock;
info_lstocks = names(raw1[2:end]);
# create a list of stocks from the livestocks array
lstocks = lstocks_array[1:end,1];
# create a NamedArray that specifies the information of each livestock
lstocks = convert(Array{Symbol},lstocks )
lstocks_matrix = lstocks_array[1:end,2:end] ;# rows are livestocks, columns are information of livestocks
lstocks_information_array = NamedArray(lstocks_matrix, (lstocks, info_lstocks), ("lstocks","info_lstocks"));

### 3.2 v1: Toy Model ###
As described in the Math Model section our first iteration was a, relatively, simple linear program which attempted to maximize net return on our farm.  This was modeled with the following code and solved with CLP:

In [16]:
#the properties of our farm
tt_field=900;
tt_housing=3;
tt_money=100000
using JuMP, Clp
m = Model(solver=ClpSolver()) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, lstock[lstocks]>=0)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)
#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,:Pasture] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,:Enclosed_Housing] for j in lstocks) <= tt_housing )

#constraint for money available
@constraint(m, sum(crop[i]*crops_information_array[i,:Cost] for i in crops) + sum(lstock[j]*lstocks_information_array[j,:Cost] for j in lstocks) <= tt_money)

#expression for the total cost of livestock
@expression(m, tot_cost_lstock, sum(lstock[i] * lstocks_information_array[i,:Cost] for i in lstocks))
#expression for the total cost of crop
@expression(m,   tot_cost_crop,sum(crop[i] * crops_information_array[i,:Cost] for i in crops))
#expression for the total return from livestock
@expression(m, gross_return_lstock, sum(lstock[i]* lstocks_information_array[i,:Return ] for i in lstocks))
#expression for the total return from crops
@expression(m, gross_return_crop, sum(crop[i]* crops_information_array[i,:Return] for i in crops))

#objective: to maximize the return
@objective(m, Max, gross_return_crop + gross_return_lstock - tot_cost_lstock - tot_cost_crop)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))

LoadError: [91mMethodError: no method matching *(::JuMP.Variable, ::Nullable{WeakRefString{UInt8}})[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *([91m::Nullable{Union{}}[39m, ::Nullable{S}) where S at /nobackup/julia/JuliaPro-0.6.2.2/JuliaPro/pkgs-0.6.2.2/v0.6/NullableArrays/src/operators.jl:108
  *(::JuMP.Variable, [91m::JuMP.Variable[39m) at /nobackup/julia/JuliaPro-0.6.2.2/JuliaPro/pkgs-0.6.2.2/v0.6/JuMP/src/operators.jl:56
  ...[39m

### 3.3 v2: Diversity Constraint###

The second version of the model added a diversity constraint.  This required adding binary indicator variables $y_i$ and $z_i$ which were set, using an upper bound, to indicate whether or not $crop_i$ or $livestock_i$ was chosen.  This was followed by adding a constraint such that the sum of $y_i$s was equal to $numcrop$, or desired number of crops, and the sum of $z_i$s was equal to numlstock, or desired number of types of animals.

In [ ]:
#the properties of our farm
tt_field=900;
tt_housing=3;
tt_money=10000000
numcrop=5
numlstock=5
lb_crops = 
lb_lstock = 

using JuMP,Gurobi
m = Model(solver=GurobiSolver(OutputFlag=0)) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, y[crops], Bin)
@variable(m, lstock[lstocks]>=0)
@variable(m, z[lstocks], Bin)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)

#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,:Pasture] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,:Enclosed_Housing] for j in lstocks) <= tt_housing )

#constraint for money available
@constraint(m, sum(crop[i]*crops_information_array[i,:Cost] for i in crops) + sum(lstock[j]*lstocks_information_array[j,:Cost] for j in lstocks) <= tt_money)

# if crop[i]>0, then y[i]=1
@constraint(m,div1[i in crops], crop[i]<=y[i]*tt_field)

# if lstock[i]>0, then z[i]=1
@constraint(m, div2[j in lstocks], lstock[j]*lstocks_information_array[j,:Cost]<=z[j]*tt_money)

#diversity constraint for crop
@constraint(m, sum(y[i] for i in crops)==numcrop)

#diversity constraint for livestock
@constraint(m,sum(z[j]for j in lstocks)>=numlstock)

#expression for the total cost of livestock
@expression(m, tot_cost_lstock, sum(lstock[i] * lstocks_information_array[i,:Cost] for i in lstocks))
#expression for the total cost of crop
@expression(m,   tot_cost_crop,sum(crop[i] * crops_information_array[i,:Cost] for i in crops))
#expression for the total return from livestock
@expression(m, gross_return_lstock, sum(lstock[i]* lstocks_information_array[i,:Return] for i in lstocks))
#expression for the total return from crops
@expression(m, gross_return_crop, sum(crop[i]* crops_information_array[i,:Return] for i in crops))

#objective: to maximize the return
@objective(m, Max, gross_return_crop + gross_return_lstock - tot_cost_lstock - tot_cost_crop)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))
println(getvalue(y))
println(getvalue(z))


### 3.4 v3: Adding Balance ###


In [ ]:
#the properties of our farm
tt_field=900;
tt_housing=90000;
tt_money=500000
numcrop=5
numlstock=5
lb_crops =tt_field /(2*numcrop)
lb_lstocks =2


using JuMP,Gurobi
m = Model(solver=GurobiSolver(OutputFlag=0)) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, y[crops], Bin)
@variable(m, lstock[lstocks], Int)
@variable(m, z[lstocks], Bin)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)

#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,:Pasture] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,:Enclosed_Housing] for j in lstocks) <= tt_housing )

#constraint for money available
@constraint(m, sum(crop[i]*crops_information_array[i,:Cost] for i in crops) + sum(lstock[j]*lstocks_information_array[j,:Cost] for j in lstocks) <= tt_money)

# if crop[i]>0, then y[i]=1
@constraint(m,div1[i in crops], crop[i]<=y[i]*tt_field)


# if lstock[i]>0, then z[i]=1
@constraint(m, div2[j in lstocks], lstock[j]*lstocks_information_array[j,:Cost]<=z[j]*tt_money)

#if y[i]=1, then crop[i]>lbcrops
@constraint(m,lb1[i in crops], crop[i]>=y[i]*lb_crops)

#if z[i]=1, then lstock[i]>lblstocks
@constraint(m,lb2[i in lstocks], lstock[i]>=z[i]*lb_lstocks)

#diversity constraint for crop
@constraint(m, sum(y[i] for i in crops)>=numcrop)

#diversity constraint for livestock
@constraint(m,sum(z[j]for j in lstocks)>=numlstock)

#expression for the total cost of livestock
@expression(m, tot_cost_lstock, sum(lstock[i] * lstocks_information_array[i,:Cost] for i in lstocks))
#expression fpr the total cost of crop
@expression(m,   tot_cost_crop,sum(crop[i] * crops_information_array[i,:Cost] for i in crops))
#expression for the total return from livestocks
@expression(m, gross_return_lstock, sum(lstock[i]* lstocks_information_array[i,:Return ] for i in lstocks))
#expression fpr the total return from crops
@expression(m, gross_return_crop, sum(crop[i]* crops_information_array[i,:Return] for i in crops))

#objective: to maximize the return
@objective(m, Max, gross_return_crop + gross_return_lstock - tot_cost_lstock - tot_cost_crop)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))
println(getvalue(y))
println(getvalue(z))

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

 Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are          | Cool  |
| ------------- |:-------------| -----:|
| col 3 is      |right-aligned |\$1600 |
|  colons       | align columns|  \$12 |
| zebra stripes |    are neat  |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.